<a href="https://colab.research.google.com/github/MikaelSM1311/BIODATA-DIRI/blob/main/Minggu_ke_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install pyrsgis

In [3]:
import os
import numpy as np
from tensorflow import keras
from pyrsgis import raster
from pyrsgis.convert import changeDimension
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, precision_score, recall_score

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
# Ubah direktori
os.chdir('/content/drive/MyDrive/Foto M6')

In [6]:
# Tetapkan nama file
mxBangalore = 'l5_Bangalore2011_raw.tif'
buildtupBangalore = 'l5_Bangalore2011_builtup.tif'
mxHyderabad = 'l5_Hyderabad2011_raw.tif'

In [7]:
# Mengakses variabel di sini dengan nama yang diperbarui
ds1, featuresBangalore = raster.read(mxBangalore, bands='all')
ds2, labelBangalore = raster.read(buildtupBangalore, bands=1)
ds3, featuresHyderabad = raster.read(mxHyderabad, bands='all')

In [8]:
# Cetak ukuran array
print('Bangalore Multispectal image shape:', featuresBangalore.shape)
print('Bangalore Binary built-up image shape:', labelBangalore.shape)
print('Hyderabad Multispectal image shape:', featuresHyderabad.shape)

Bangalore Multispectal image shape: (6, 2054, 2044)
Bangalore Binary built-up image shape: (2054, 2044)
Hyderabad Multispectal image shape: (6, 1318, 1056)


In [9]:
# Bersihkan data berlabel untuk mengganti nilai NoData dengan nol
labelBangalore = (labelBangalore == 1).astype(int)

# Ubah bentuk array menjadi array berdimensi tunggal
featuresBangalore = changeDimension(featuresBangalore)
labelBangalore = changeDimension(labelBangalore)
featuresHyderabad = changeDimension(featuresHyderabad)
nBands = featuresBangalore.shape[1]

print('Bangalore Multispectal image shape:', featuresBangalore.shape)
print('Bangalore Binary built-up image shape:', labelBangalore.shape)
print('Hyderabad Multispectal image shape:', featuresHyderabad.shape)

The "changeDimension()" function has moved to "array_to_table()" and will be deprecated in future versions. Please check the pyrsgis documentation at https://pyrsgis.readthedocs.io/en/master/ for more details.
The "changeDimension()" function has moved to "array_to_table()" and will be deprecated in future versions. Please check the pyrsgis documentation at https://pyrsgis.readthedocs.io/en/master/ for more details.
The "changeDimension()" function has moved to "array_to_table()" and will be deprecated in future versions. Please check the pyrsgis documentation at https://pyrsgis.readthedocs.io/en/master/ for more details.
Bangalore Multispectal image shape: (4198376, 6)
Bangalore Binary built-up image shape: (4198376,)
Hyderabad Multispectal image shape: (1391808, 6)


In [10]:
# Pengujian tumpahan dan pelatihan databest
xTrain, xTest, yTrain, yTest = train_test_split (featuresBangalore, labelBangalore, test_size=0.4, random_state=42)

print ("xTrain shape:", xTrain.shape)
print ("yTrain shape:", yTrain.shape)

# Normalisasikan data
xTrain = xTrain / 255.0
xTest = xTest / 255.0
featuresHyderabad = featuresHyderabad / 255.0

# Membentuk ulang data
xtrain = xTrain.reshape((xTrain.shape[0], 1, xTrain.shape[1]))
xtest = xTest.reshape((xTest.shape[0], 1, xTest.shape[1]))

featuresHyderabad = featuresHyderabad.reshape((featuresHyderabad.shape[0], 1, xTrain.shape[1]))
print(xtrain.shape, xtest.shape, featuresHyderabad.shape)

xTrain shape: (2519025, 6)
yTrain shape: (2519025,)
(2519025, 1, 6) (1679351, 1, 6) (1391808, 1, 6)


In [11]:
# Tentukan parameter modelnya
model = keras.Sequential ([
    keras.layers.Flatten(input_shape=(nBands,)), #modified line - removed extra dimension from input shape
    keras.layers.Dense(10, activation = 'relu'),
    keras.layers.Dense(5, activation = 'relu'),
    keras.layers.Dense(2, activation = 'softmax')])

# Tentukan metrik dan parameter akurasi
model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

model.fit(xTrain, yTrain, epochs=2)

# Prediksi untuk data uji
yTestPredicted = model.predict(xTest)
yTestPredicted = yTestPredicted[:,1]

# Hitung dan tampilkan metrik kesalahan
yTestPredicted = (yTestPredicted>0.5).astype(int)
cMatrix = confusion_matrix(yTest, yTestPredicted)
pScore = precision_score(yTest, yTestPredicted)
rScore = recall_score(yTest, yTestPredicted)

print("Confusion Matrix: for 14 nodes\n", cMatrix)
print("\nP-Score: %.3f, R-Score: %.3f" % (pScore, rScore))

Epoch 1/2
78720/78720 ━━━━━━━━━━━━━━━━━━━━ 172s 2ms/step - accuracy: 0.9488 - loss: 0.1392
Epoch 2/2
78720/78720 ━━━━━━━━━━━━━━━━━━━━ 188s 2ms/step - accuracy: 0.9592 - loss: 0.1083
52480/52480 ━━━━━━━━━━━━━━━━━━━━ 62s 1ms/step
Confusion Matrix: for 14 nodes
 [[1455280   25596]
 [  42066  156409]]

P-Score: 0.859, R-Score: 0.788


In [18]:
# Melakukan prediksi pada data Hyderabad
predicted = model.predict(featuresHyderabad)
predicted = predicted[:,1]

# Prediksi data baru dan ekspor hasil sebagai raster probabilitas
prediction = np.reshape(predicted, (ds3.RasterYSize, ds3.RasterXSize)) # Changed dst3 to ds3
outfile = 'Hyderabad_2011_BuiltupNN_predicted.tif'
raster.export(prediction, ds3, filename=outfile, dtype='float') # Changed dst3 to ds3

43494/43494 ━━━━━━━━━━━━━━━━━━━━ 52s 1ms/step


In [24]:
# Menyiapkan data untuk SVM
xTrain2D = np.zeros((len(xTrain), 6))
xTest2D = np.zeros((len(xTest), 6))
featuresHyderabad2D = np.zeros((len(featuresHyderabad), 6))

for i in range(6):
    xTrain2D[:, i] = xTrain[:, i]
    xTest2D[:, i] = xTest[:, i]
    featuresHyderabad2D[:, i] = featuresHyderabad[:, i]

random_indices = np.random.choice(len(xTrain2D), size=1000, replace=False)
xTrain2D_random = xTrain2D[random_indices, :]
yTrain_random = yTrain[random_indices]

In [25]:
# Memproses SVM
from sklearn import svm
clf = svm.SVC()
clf.fit(xTrain2D_random, yTrain_random)

predicted = clf.predict(featuresHyderabad2D)

In [26]:
# Prediksi hasil dan ekspor raster probablilitas
prediction = np.reshape(predicted, (ds3.RasterYSize, ds3.RasterXSize))
outfile = 'Hyderabad_2011_BuiltupSVM_predicted.tif'
raster.export(prediction, ds3, filename=outfile, dtype='float')

# Prediksi untuk data uji
yTestPredicted_SVM = clf.predict(xTest2D)

In [27]:
# Perhitungan matriks error dari hasil ekstraksi
CMatrix = confusion_matrix(yTest, yTestPredicted_SVM)
pScore = precision_score(yTest, yTestPredicted_SVM)
rScore = recall_score(yTest, yTestPredicted_SVM)

print('Confusion matrix: for SVM/n', cMatrix)
print('\nP-Score: %.3f, R-Score: %.3f' % (pScore, rScore))

Confusion matrix: for SVM/n [[1455280   25596]
 [  42066  156409]]

P-Score: 0.899, R-Score: 0.657
